In [1]:
from pdb import Restart
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torch.utils.data
import os
import argparse
from tqdm import tqdm
import argparse
import collections
import json
import random
import time
import uuid

sys.path.append('../../../')

import numpy as np
import PIL

from domainbed import datasets
from domainbed import hparams_registry
from domainbed import algorithms
from domainbed.lib import misc
from domainbed.lib.fast_data_loader import InfiniteDataLoader, FastDataLoader

import matplotlib.pyplot as plt
import shap
import matplotlib as mpl
from captum.attr import LayerGradCam

sys.path.append('../../')
from datasets import make_dataset
from mixuploss import MixupLoss
from networks import ResNet

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
parser = argparse.ArgumentParser(description='Domain generalization')
parser.add_argument('--data_dir', type=str)
parser.add_argument('--dataset', type=str, default="RotatedMNIST")
parser.add_argument('--algorithm', type=str, default="ERM")
parser.add_argument('--task', type=str, default="domain_generalization",
    choices=["domain_generalization", "domain_adaptation"])
parser.add_argument('--hparams', type=str,
    help='JSON-serialized hparams dict')
parser.add_argument('--hparams_seed', type=int, default=0,
    help='Seed for random hparams (0 means "default hparams")')
parser.add_argument('--trial_seed', type=int, default=0,
    help='Trial number (used for seeding split_dataset and '
    'random_hparams).')
parser.add_argument('--seed', type=int, default=0,
    help='Seed for everything else')
parser.add_argument('--steps', type=int, default=None,
    help='Number of steps. Default is dataset-dependent.')
parser.add_argument('--checkpoint_freq', type=int, default=None,
    help='Checkpoint every N steps. Default is dataset-dependent.')
parser.add_argument('--test_envs', type=int, nargs='+', default=[0])
parser.add_argument('--output_dir', type=str, default="train_output")
parser.add_argument('--holdout_fraction', type=float, default=0.2)
parser.add_argument('--uda_holdout_fraction', type=float, default=0,
    help="For domain adaptation, % of test to use unlabeled for training.")
parser.add_argument('--skip_model_save', action='store_true')
parser.add_argument('--save_model_every_checkpoint', action='store_true')

In [7]:
args = parser.parse_args(args=['--data_dir', '../../../data/',
                               '--algorithm', 'GroupDRO',
                               '--dataset', 'TerraIncognita',
                               '--test_env', '3',
                               '--output_dir', '../visualizations/outputs/'])

In [ ]:
# If we ever want to implement checkpointing, just persist these values
# every once in a while, and then load them from disk here.
start_step = 0
algorithm_dict = None

print("Environment:")
print("\tPython: {}".format(sys.version.split(" ")[0]))
print("\tPyTorch: {}".format(torch.__version__))
print("\tTorchvision: {}".format(torchvision.__version__))
print("\tCUDA: {}".format(torch.version.cuda))
print("\tCUDNN: {}".format(torch.backends.cudnn.version()))
print("\tNumPy: {}".format(np.__version__))
print("\tPIL: {}".format(PIL.__version__))

print('Args:')
for k, v in sorted(vars(args).items()):
    print('\t{}: {}'.format(k, v))

if args.hparams_seed == 0:
    hparams = hparams_registry.default_hparams(args.algorithm, args.dataset)
else:
    hparams = hparams_registry.random_hparams(args.algorithm, args.dataset,
        misc.seed_hash(args.hparams_seed, args.trial_seed))
if args.hparams:
    hparams.update(json.loads(args.hparams))

print('HParams:')
for k, v in sorted(hparams.items()):
    print('\t{}: {}'.format(k, v))

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [9]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

if args.dataset in vars(datasets):
    dataset = vars(datasets)[args.dataset](args.data_dir,
        args.test_envs, hparams)
else:
    raise NotImplementedError

In [ ]:
# Split each env into an 'in-split' and an 'out-split'. We'll train on
# each in-split except the test envs, and evaluate on all splits.

# To allow unsupervised domain adaptation experiments, we split each test
# env into 'in-split', 'uda-split' and 'out-split'. The 'in-split' is used
# by collect_results.py to compute classification accuracies.  The
# 'out-split' is used by the Oracle model selectino method. The unlabeled
# samples in 'uda-split' are passed to the algorithm at training time if
# args.task == "domain_adaptation". If we are interested in comparing
# domain generalization and domain adaptation results, then domain
# generalization algorithms should create the same 'uda-splits', which will
# be discared at training.
in_splits = []
out_splits = []
uda_splits = []
for env_i, env in enumerate(dataset):
    uda = []

    out, in_ = misc.split_dataset(env,
        int(len(env)*args.holdout_fraction),
        misc.seed_hash(args.trial_seed, env_i))

    if env_i in args.test_envs:
        uda, in_ = misc.split_dataset(in_,
            int(len(in_)*args.uda_holdout_fraction),
            misc.seed_hash(args.trial_seed, env_i))

    if hparams['class_balanced']:
        in_weights = misc.make_weights_for_balanced_classes(in_)
        out_weights = misc.make_weights_for_balanced_classes(out)
        if uda is not None:
            uda_weights = misc.make_weights_for_balanced_classes(uda)
    else:
        in_weights, out_weights, uda_weights = None, None, None
    in_splits.append((in_, in_weights))
    out_splits.append((out, out_weights))
    if len(uda):
        uda_splits.append((uda, uda_weights))

if args.task == "domain_adaptation" and len(uda_splits) == 0:
    raise ValueError("Not enough unlabeled samples for domain adaptation.")

train_loaders = [InfiniteDataLoader(
    dataset=env,
    weights=env_weights,
    batch_size=hparams['batch_size'],
    num_workers=dataset.N_WORKERS)
    for i, (env, env_weights) in enumerate(in_splits)
    if i not in args.test_envs]

uda_loaders = [InfiniteDataLoader(
    dataset=env,
    weights=env_weights,
    batch_size=hparams['batch_size'],
    num_workers=dataset.N_WORKERS)
    for i, (env, env_weights) in enumerate(uda_splits)
    if i in args.test_envs]

eval_loaders = [FastDataLoader(
    dataset=env,
    batch_size=64,
    num_workers=dataset.N_WORKERS)
    for env, _ in (in_splits + out_splits + uda_splits)]
    
eval_weights = [None for _, weights in (in_splits + out_splits + uda_splits)]

eval_loader_names = ['env{}_in'.format(i)
    for i in range(len(in_splits))]
eval_loader_names += ['env{}_out'.format(i)
    for i in range(len(out_splits))]
eval_loader_names += ['env{}_uda'.format(i)
    for i in range(len(uda_splits))]

test_loaders = [eval_loaders[0], eval_loaders[4]]

algorithm_class = algorithms.get_algorithm_class(args.algorithm)
algorithm = algorithm_class(dataset.input_shape, dataset.num_classes,
    len(dataset) - len(args.test_envs), hparams)

if algorithm_dict is not None:
    algorithm.load_state_dict(algorithm_dict)

algorithm.to(device)


train_minibatches_iterator = zip(*train_loaders)
uda_minibatches_iterator = zip(*uda_loaders)
checkpoint_vals = collections.defaultdict(lambda: [])

steps_per_epoch = min([len(env)/hparams['batch_size'] for env,_ in in_splits])

n_steps = args.steps or dataset.N_STEPS
checkpoint_freq = args.checkpoint_freq or dataset.CHECKPOINT_FREQ

test_minibatches_iterator = zip(*test_loaders)

minibatches_device = [(x.to(device), y.to(device))
        for x,y in next(train_minibatches_iterator)]
uda_device = None
step_vals = algorithm.update(minibatches_device, uda_device)

minibatches_device = [(x.to(device), y.to(device))
        for x,y in next(train_minibatches_iterator)]
uda_device = None
step_vals = algorithm.update(minibatches_device, uda_device)

In [ ]:
checkpoint = torch.load('../../../domainbed/train_output/IRM/best_model.pkl')
algorithm.load_state_dict(checkpoint['model_dict'])
net_base = algorithm.network.to(device)
net_base.eval()

In [ ]:
net_ours = ResNet()
net_ours = nn.DataParallel(net_ours)
net_ours.module.network.fc = nn.Linear(net_ours.module.network.fc.in_features, 10)
net_ours.load_state_dict(torch.load('../../ckpts/best_model.pth'))
net_ours = net_ours.to(device)
net_ours.eval()

In [13]:
transform = transforms.Compose([
            # transforms.Resize((224,224)),
            transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(0.3, 0.3, 0.3, 0.3),
            transforms.RandomGrayscale(),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

transform_orig = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor()
        ])

In [14]:
dset38 = torchvision.datasets.ImageFolder('../../../data/terra_incognita/location_38/', transform=transform)
dset43 = torchvision.datasets.ImageFolder('../../../data/terra_incognita/location_43/', transform=transform)
dset46 = torchvision.datasets.ImageFolder('../../../data/terra_incognita/location_46/', transform=transform)
dset100 = torchvision.datasets.ImageFolder('../../../data/terra_incognita/location_100/', transform=transform)

dset38_orig = torchvision.datasets.ImageFolder('../../../data/terra_incognita/location_38/', transform=transform_orig)
dset43_orig = torchvision.datasets.ImageFolder('../../../data/terra_incognita/location_43/', transform=transform_orig)
dset46_orig = torchvision.datasets.ImageFolder('../../../data/terra_incognita/location_46/', transform=transform_orig)
dset100_orig = torchvision.datasets.ImageFolder('../../../data/terra_incognita/location_100/', transform=transform_orig)

In [15]:
class_names = np.sort(dset38.classes)

In [16]:
class_names

array(['bird', 'bobcat', 'cat', 'coyote', 'dog', 'empty', 'opossum',
       'rabbit', 'raccoon', 'squirrel'], dtype='<U8')

In [17]:
bs = 16
imgsz = 224

In [18]:
loader38 = torch.utils.data.DataLoader(dset38, batch_size=bs, shuffle=False, num_workers=8)
loader43 = torch.utils.data.DataLoader(dset43, batch_size=bs, shuffle=False, num_workers=8)
loader46 = torch.utils.data.DataLoader(dset46, batch_size=bs, shuffle=False, num_workers=8)
loader100 = torch.utils.data.DataLoader(dset100, batch_size=bs, shuffle=False, num_workers=8)

loader38_orig = torch.utils.data.DataLoader(dset38_orig, batch_size=bs, shuffle=False, num_workers=8)
loader43_orig = torch.utils.data.DataLoader(dset43_orig, batch_size=bs, shuffle=False, num_workers=8)
loader46_orig = torch.utils.data.DataLoader(dset46_orig, batch_size=bs, shuffle=False, num_workers=8)
loader100_orig = torch.utils.data.DataLoader(dset100_orig, batch_size=bs, shuffle=False, num_workers=8)

In [19]:
layer_gc_base = LayerGradCam(net_base, net_base[0].network.layer4)
layer_gc_ours = LayerGradCam(net_ours, net_ours.module.network.layer4)

In [20]:
def attr_scale(x):
    return (x-x.min()) / (x.max()-x.min())

def explain(image, image_orig, pred_base, pred_ours, count, imgsz=224, target_env='Caltech101', mode='ood'):
    image = image.reshape(1, 3, imgsz, imgsz).to(device)
    image_orig = image_orig.reshape(1, 3, imgsz, imgsz).to(device)

    attr_base = layer_gc_base.attribute(image, target=pred_base)
    attr_ours = layer_gc_ours.attribute(image, target=pred_ours)

    attribution_base = F.interpolate(attr_base, size=imgsz, mode='bilinear').squeeze()
    attribution_ours = F.interpolate(attr_ours, size=imgsz, mode='bilinear').squeeze()

    attribution_base = attr_scale(attribution_base)
    attribution_ours = attr_scale(attribution_ours)

    cmap = mpl.cm.get_cmap('jet', 256)
    heatmap_base = cmap(attribution_base.cpu().detach().numpy(), alpha = 0.5)
    heatmap_ours = cmap(attribution_ours.cpu().detach().numpy(), alpha = 0.5)


    fig, ax = plt.subplots(1, 3, figsize=(4, 2), dpi=200)
    # fig.suptitle('Grad-CAM', fontsize=20)

    ax[0].imshow(image_orig.squeeze().cpu().detach().numpy().transpose(1, 2, 0))
    ax[0].set_title(str(mode) + ' ' + str(target_env))
    ax[0].axis('off')

    ax[1].imshow(image_orig.squeeze().cpu().detach().numpy().transpose(1, 2, 0))
    ax[1].set_title('GroupDRO')
    ax[1].axis('off')
    ax[2].imshow(image_orig.squeeze().cpu().detach().numpy().transpose(1, 2, 0))
    ax[2].set_title('ours')
    ax[2].axis('off')

    ax[1].imshow(heatmap_base)
    ax[1].axis('off')
    ax[2].imshow(heatmap_ours)
    ax[2].axis('off')

    fig.savefig('./dro/{}/{}/{}/{}_{}.jpeg'.format(mode, target_env, str(class_names[int(pred_ours.cpu().detach().numpy())]), str(count), str(class_names[int(pred_ours.cpu().detach().numpy())])))
    plt.close(fig)

In [ ]:
count = 0
it = iter(loader46)
it_orig = iter(loader46_orig)

for batch_idx in tqdm(range(len(it))):
    inputs, targets = next(it)
    inputs_orig, _ = next(it_orig)

    inputs = inputs.to(device)
    targets = targets.to(device)

    outputs_base = net_base(inputs)
    outputs_ours = net_ours(inputs)

    preds_base = outputs_base.argmax(dim=1)
    preds_ours = outputs_ours.argmax(dim=1)

    for i in range(len(inputs)):
        if targets[i] == 3:
            if preds_ours[i] == targets[i]:
                explain(inputs[i], inputs_orig[i], preds_base[i], preds_ours[i], count, target_env='L46', mode='ood')
        count += 1